# Segmenting and Clustering Neighborhoods in Toronto, Canada

In [1]:
import numpy as np #Library to handle data in a vectorized manner

In [2]:
import pandas as pd #Library for data analysis


In [3]:
import json #Library to handle JSON files

In [4]:
from geopy.geocoders import Nominatim #convert an address to latitude and longitude values

In [5]:
import requests #Library to handle requests

In [6]:
from pandas.io.json import json_normalize #transform JSON file into pandas dataframe

In [7]:
import matplotlib.cm as cm #Matplotlib and associated plotting modules
import matplotlib.colors as colors

In [8]:
from sklearn.cluster import KMeans #import k-means from clustering stage

## 1.Scrapping the web page using BeautifulSoup Package

* Creating a DataFrame with threee columns: PostalCode, Borough and Neighborhood

In [9]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text #getting the list of postal codes from wikipedia

In [10]:
from bs4 import BeautifulSoup # using the BeautifulSoup package to scrap the data
soup = BeautifulSoup(source, 'lxml')

In [11]:
table = soup.find('table', class_='wikitable sortable' )

In [12]:
Headers = ['PostalCode', 'Borough', 'Neighborhood'] # creating the dataframe with three required coumns
df = pd.DataFrame(columns = Headers)


In [13]:
for tr in table.find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
        if len(row)==3:
            df.loc[len(df)] = row 
        

In [14]:
df.head() # showing top 5 valus of the dataframe

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


* Ignoring cells with a Borough that is **Not Assigned**

In [15]:
df = df.drop(df[(df.Borough == 'Not assigned')].index)# dropping a row
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


* If a cell has a borough but a **Not Assigned** neighborhood, then the neighborhood will be same as the borough

In [16]:
df.Neighborhood.replace("Not assigned", df.Borough, inplace = True)


* Combining all neighborhoods(seperated with a comma) in one PostalCode into one row

In [17]:
df = df.groupby(['PostalCode', 'Borough'], sort = False).agg(','.join)
df = df.reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
df.shape # returns the number of rows and columns in the dataframe

(103, 3)

## 2. Adding latitude and longitude values to the dataframe

In [19]:
LL_toronto = pd.read_csv("https://cocl.us/Geospatial_data")#Reading the values from the csv file
LL_toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
LL_toronto.rename(columns = {'Postal Code' : 'PostalCode'}, inplace = True) #Renaming Postal Code to PostalCode
LL_toronto.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
Toronto = pd.merge(df, LL_toronto, on = 'PostalCode') #Merging two dataframes on PostalCode value
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3.Exploring and Clustering the neighborhoods in Toronto

In [22]:
from geopy.geocoders import Nominatim #Convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode("Toronto")
LATITUDE = location.latitude
LONGITUDE = location.longitude
print('The geographical Coordinates of Toronto are {}, {}.'.format(LATITUDE,LONGITUDE))


The geographical Coordinates of Toronto are 43.6534817, -79.3839347.


### Generating Toronto Map using Folium

In [23]:
!pip install folium #map rendering library
import folium

In [24]:
map_toronto = folium.Map(location=[LATITUDE,LONGITUDE], zoom_start = 10) # Generating Map of Toronto
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
    
map_toronto    
    

* Let us only consider neighborhoods which contains the string "Toronto" and create a new dataframe

In [25]:
Toronto_data = Toronto[Toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Generating Map of Boroughs containing string "Toronto"

In [26]:
geolocator = Nominatim(user_agent = "to_explorer")
location = geolocator.geocode('Toronto, ON')
LATITUDE1 = location.latitude
LONGITUDE2 = location.longitude
print('The Geographical Coordinates of Toronto are {},{}.'.format(LATITUDE1,LONGITUDE2))

The Geographical Coordinates of Toronto are 43.6534817,-79.3839347.


In [27]:
map_Toronto= folium.Map(location=[LATITUDE1,LONGITUDE2], zoom_start = 10)
for lat, lng,  label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'],Toronto_data['Neighborhood']):
    
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Toronto)
    
    
map_Toronto  

## Defining FourSquare Credentials and Version

In [28]:
CLIENT_ID = 'YCSCUTEWXS2BWBQJ4UMBQX1EN4SPLSA2BXTZ0RNJ40MHFRO4'
CLIENT_SECRET = 'OD05LMRBSQXQF05BDGSGDY41JSROAEMJ5EK4HULIZ4YE3H1G'
VERSION = '20180605'
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: YCSCUTEWXS2BWBQJ4UMBQX1EN4SPLSA2BXTZ0RNJ40MHFRO4
CLIENT_SECRET:OD05LMRBSQXQF05BDGSGDY41JSROAEMJ5EK4HULIZ4YE3H1G


* Exploring **First** Neighborhood in Toronto_data

In [29]:
Toronto_data.loc[0,'Neighborhood']

'Regent Park, Harbourfront'

In [30]:
neighborhood_latitude = Toronto_data.loc[0,'Latitude']
neighborhood_longitude = Toronto_data.loc[0,'Longitude']
print('Latitude and Longitude values are {}, {}.'.format(neighborhood_latitude,neighborhood_longitude))

Latitude and Longitude values are 43.6542599, -79.3606359.


* Let us explore top 100 venues in the selected Neighborhood

In [31]:
limit = 100 # Limiting the number of venues to 100
radius = 500 # Limiting the radius to 500 mts
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit{}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YCSCUTEWXS2BWBQJ4UMBQX1EN4SPLSA2BXTZ0RNJ40MHFRO4&client_secret=OD05LMRBSQXQF05BDGSGDY41JSROAEMJ5EK4HULIZ4YE3H1G&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit100'

In [32]:
results = requests.get(url).json()
#results

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
            

* Cleaning the json and structuring it into pandas dataframe

In [34]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis =1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [35]:
print('{} venues returned by Foursquare.'. format(nearby_venues.shape[0]))

30 venues returned by Foursquare.


* Exploring **all** Neighborhood in Toronto_data

In [36]:
def getNearbyVenues(names, latitudes,longitudes, radius = 500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes,longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit{}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            limit)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in  results])
     
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)
                             
            

In [37]:
Toronto_venues =  getNearbyVenues(names=Toronto_data['Neighborhood'],
                                          latitudes = Toronto_data['Latitude'],
                                          longitudes = Toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [38]:
print(Toronto_venues.shape)
Toronto_venues.head()

(864, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [39]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [40]:
print('There are {} unique categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 191 unique categories.


## Analyzing each Neighborhood

In [41]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix ="", prefix_sep="")#One hot encoding
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']#Adding back the Neighborhood column
fixed_columns = [Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1]) #Mving Neighborhood column to the first column
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
Toronto_onehot.shape

(864, 191)

In [43]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index() # taking the mean of the frequency of occurance of each category
Toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [44]:
Toronto_grouped.shape

(39, 191)

* Finding the top venues

In [45]:
num_top_venues = 5
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(num_top_venues))

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2         Coffee Shop  0.07
3            Beer Bar  0.07
4        Concert Hall  0.03
----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.13
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.04
4             Stadium  0.04
----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0         Yoga Studio  0.06
1             Brewery  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06
----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4               Bar  0.06
----Central Bay Street----
                       venue  fr

* Finding top 10 venues

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10 #Putting into a Dtaframe

indicators = ['st','nd','rd']

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()    

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Beer Bar,Coffee Shop,Seafood Restaurant,Jazz Club,Basketball Stadium,Farmers Market,Bistro,Concert Hall,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Convenience Store,Climbing Gym,Restaurant,Burrito Place
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Light Rail Station,Brewery,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Bar
4,Central Bay Street,Coffee Shop,Café,Park,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Chinese Restaurant,Yoga Studio


# 4.Clustering Neighborhoods

In [48]:
kclusters = 5 #Number of clusters
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood',1)
Kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(Toronto_grouped_clustering)#K-means clustering
Kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [49]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels', Kmeans.labels_)
Toronto_merged = Toronto_data
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')


In [51]:
Toronto_merged


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Historic Site,Ice Cream Shop,Farmers Market,Event Space,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Wings Joint,Distribution Center,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Gym
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Coffee Shop,Theater,Ramen Restaurant,Bar,Hotel,Shopping Mall,Sandwich Place,Diner,Burger Joint
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Gastropub,Coffee Shop,Cocktail Bar,New American Restaurant,Middle Eastern Restaurant,Restaurant,Jazz Club,Japanese Restaurant,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Wings Joint,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Cocktail Bar,Beer Bar,Coffee Shop,Seafood Restaurant,Jazz Club,Basketball Stadium,Farmers Market,Bistro,Concert Hall,Breakfast Spot
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Park,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Chinese Restaurant,Yoga Studio
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Restaurant,Coffee Shop,Nightclub,Diner,Italian Restaurant,Candy Store,Baby Store
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Colombian Restaurant,Deli / Bodega,Hotel,Smoke Shop
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Pharmacy,Bakery,Café,Bank,Supermarket,Brewery,Pool,Middle Eastern Restaurant,Bar,Furniture / Home Store


* Geneating Map of Clusters

In [52]:
map_clusters = folium.Map(location=[LATITUDE1,LONGITUDE2], zoom_start = 12)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+' Cluster ' + str(cluster), parse_html =True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup = label,
    color = rainbow[int(cluster-1)],
    fill = True,
    fill_color = rainbow[int(cluster-1)],
    fill_opacity = 0.7).add_to(map_clusters)    
        
map_clusters    

## Examining Clusters

In [53]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Historic Site,Ice Cream Shop,Farmers Market,Event Space,Distribution Center
1,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Wings Joint,Distribution Center,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Gym
6,Downtown Toronto,0,Coffee Shop,Café,Park,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Chinese Restaurant,Yoga Studio
25,West Toronto,0,Breakfast Spot,Gift Shop,Bookstore,Coffee Shop,Eastern European Restaurant,Dog Run,Cuban Restaurant,Movie Theater,Bar,Italian Restaurant


In [54]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Café,Coffee Shop,Theater,Ramen Restaurant,Bar,Hotel,Shopping Mall,Sandwich Place,Diner,Burger Joint
3,Downtown Toronto,1,Café,Gastropub,Coffee Shop,Cocktail Bar,New American Restaurant,Middle Eastern Restaurant,Restaurant,Jazz Club,Japanese Restaurant,Italian Restaurant
4,East Toronto,1,Trail,Health Food Store,Pub,Wings Joint,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
5,Downtown Toronto,1,Cocktail Bar,Beer Bar,Coffee Shop,Seafood Restaurant,Jazz Club,Basketball Stadium,Farmers Market,Bistro,Concert Hall,Breakfast Spot
7,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Coffee Shop,Nightclub,Diner,Italian Restaurant,Candy Store,Baby Store
8,Downtown Toronto,1,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Colombian Restaurant,Deli / Bodega,Hotel,Smoke Shop
9,West Toronto,1,Pharmacy,Bakery,Café,Bank,Supermarket,Brewery,Pool,Middle Eastern Restaurant,Bar,Furniture / Home Store
10,Downtown Toronto,1,Park,Hotel,Plaza,Performing Arts Venue,Salad Place,IT Services,Ice Cream Shop,Skating Rink,Italian Restaurant,Japanese Restaurant
11,West Toronto,1,Asian Restaurant,Bar,Men's Store,Yoga Studio,Korean Restaurant,Ice Cream Shop,Italian Restaurant,Brewery,Japanese Restaurant,Record Shop
12,East Toronto,1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Pizza Place,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant,Pub


In [55]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Park,Trail,Restaurant,Wings Joint,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
33,Downtown Toronto,2,Park,Playground,Trail,Wings Joint,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store


In [56]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Bus Line,Swim School,Wings Joint,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center


In [57]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Garden,Home Service,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
